In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Create dataset -> DataFrame
### **Inference dataset for fashion recommendation**
---

## Define Value Pools

In [2]:
BRANDS = ["ZARA", "Adidas", "Nike", "HnM", "Polo", "Tommy Hilfiger"]

SUBCATEGORIES = {
    "tops": [
        "printed_tshirts",
        "solid_tshirts",
        "printed_hoodies",
        "casual_shirts",
        "formal_shirts",
    ],
    "bottoms": [
        "formal_pants",
        "jeans",
        "men_cargos",
    ]
}

SLEEVE_BY_SUBCATEGORY = {
    "printed_tshirts": ["short"],
    "solid_tshirts": ["short"],
    "printed_hoodies": ["long"],
    "casual_shirts": ["long"],
    "formal_shirts": ["long"],
    "formal_pants": [None],
    "jeans": [None],
    "men_cargos": [None],
}

SEASONS = ["summer", "winter", "all-season"]
FABRICS = ["cotton", "denim", "polyester", "fleece", "linen"]
OCCASIONS = ["casual", "office", "party"]
FORMALITY = ["low", "medium", "high"]
SIZES = ["XS", "S", "M", "L", "XL", "XXL"]

In [3]:
import random

# Helper Functions (Realistic Numbers)
def random_length(subcategory):
    if subcategory in ["printed_tshirts", "solid_tshirts"]:
        return random.randint(65, 75)
    if subcategory == "printed_hoodies":
        return random.randint(70, 80)
    if subcategory in ["casual_shirts", "formal_shirts"]:
        return random.randint(68, 78)
    if subcategory in ["formal_pants", "jeans", "men_cargos"]:
        return random.randint(95, 110)
    return None

def random_price(subcategory):
    base = {
        "printed_tshirts": 150_000,
        "solid_tshirts": 130_000,
        "printed_hoodies": 320_000,
        "casual_shirts": 250_000,
        "formal_shirts": 300_000,
        "formal_pants": 350_000,
        "jeans": 330_000,
        "men_cargos": 280_000,
    }
    return int(np.random.normal(base[subcategory], base[subcategory] * 0.15))

# Generate ONE Random Item (Core Logic)

In [4]:
def generate_item(idx):
    category = random.choice(list(SUBCATEGORIES.keys()))
    subcategory = random.choice(SUBCATEGORIES[category])

    return {
        "item_id": f"TNC_{idx:06d}",
        "category": category,
        "subcategory": subcategory,
        "sleeve_type": random.choice(SLEEVE_BY_SUBCATEGORY[subcategory]),
        "season": random.choice(SEASONS),
        "fabric": random.choice(FABRICS),
        "occasion": random.choice(OCCASIONS),
        "formality_level": random.choice(FORMALITY),
        "size_range": random.choice(SIZES),
        "brand": random.choice(BRANDS),
        "view_count": int(np.random.exponential(800)),
        "click_count": int(np.random.exponential(150)),
        "purchase_count": int(np.random.exponential(30)),
        "length_cm": random_length(subcategory),
        "price": random_price(subcategory),
        "stocks": random.randint(0, 200),
    }

##  Generate 1,100,000 Rows

In [5]:
from tqdm import trange

N_ROWS = 1100000
data = [generate_item(i) for i in trange(1, N_ROWS + 1)]
df = pd.DataFrame(data)

100%|██████████| 1100000/1100000 [00:03<00:00, 298158.57it/s]


## Enforce Data Types

In [6]:
NUMERIC_COLS = [
    "view_count",
    "click_count",
    "purchase_count",
    "length_cm",
    "price",
    "stocks",
]

OBJECT_COLS = [
    "item_id",
    "category",
    "subcategory",
    "sleeve_type",
    "season",
    "fabric",
    "occasion",
    "formality_level",
    "size_range",
    "brand",
]

df[NUMERIC_COLS] = df[NUMERIC_COLS].apply(pd.to_numeric, errors="coerce")
df[OBJECT_COLS] = df[OBJECT_COLS].astype("string")

In [7]:
df

,item_id,category,subcategory,sleeve_type,season,fabric,occasion,formality_level,size_range,brand,view_count,click_count,purchase_count,length_cm,price,stocks
0,TNC_000001,bottoms,men_cargos,<NA>,winter,denim,casual,low,XXL,ZARA,206,432,1,107,260868,60
1,TNC_000002,bottoms,formal_pants,<NA>,summer,cotton,party,high,L,Adidas,72,82,38,109,443568,119
2,TNC_000003,tops,formal_shirts,long,winter,fleece,party,low,M,HnM,232,225,35,73,304315,47
3,TNC_000004,tops,solid_tshirts,short,winter,polyester,casual,low,XXL,ZARA,115,63,43,65,108979,129
4,TNC_000005,bottoms,jeans,<NA>,winter,fleece,party,low,XXL,HnM,325,99,14,97,317333,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099995,TNC_1099996,bottoms,men_cargos,<NA>,winter,linen,casual,low,S,Adidas,1293,124,26,103,256913,115
1099996,TNC_1099997,tops,solid_tshirts,short,winter,cotton,office,high,XXL,ZARA,174,117,44,73,134164,117
1099997,TNC_1099998,tops,printed_hoodies,long,summer,polyester,party,low,XS,ZARA,254,179,15,78,314806,68
1099998,TNC_1099999,bottoms,jeans,<NA>,winter,cotton,party,low,S,Nike,1884,147,12,110,377387,153


In [ ]:
# Get the most common value to handle misising value
most_frequent = df['sleeve_type'].mode()[0]
df['sleeve_type'] = df['sleeve_type'].fillna(most_frequent)

In [9]:
df.isnull().sum()

item_id            0
category           0
subcategory        0
sleeve_type        0
season             0
fabric             0
occasion           0
formality_level    0
size_range         0
brand              0
view_count         0
click_count        0
purchase_count     0
length_cm          0
price              0
stocks             0
dtype: int64